In [1]:
!pip install transformers datasets

In [9]:
import os
os.environ["WANDB_DISABLED"] = "true"

from datasets import load_dataset

# Load dataset
dataset = load_dataset('json', data_files={
    'train': '../Datasets/Training_datasets/Training_dataset_simple_10k.jsonl',
    'test': '../Datasets/Training_datasets/Training_dataset_simple_3k.jsonl'
})

# Preview dataset
print("Example training data:", dataset["train"][0])
print("Example test data:", dataset["test"][0])

FileNotFoundError: Unable to find '/Users/luddesmac/Desktop/Skola/TUM/Next-gen-prog/berlin/TestingCodeT5/Datasets/Training_datasets/Training_dataset_simple_10k.jsonl'

In [ ]:
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5-base")

# Preprocessing
def preprocess_function(examples):
    inputs = tokenizer(examples["question"], max_length=128, truncation=True, padding="max_length")
    targets = tokenizer(examples["expected_code"], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

# Tokenize dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Verify tokenized dataset
print("Tokenized dataset example:", tokenized_dataset["train"][0])

Tokenized dataset example: {'question': 'What is the sum of 3 and 5?', 'expected_code': 'def solve(): return 3 + 5', 'input_ids': [1, 23801, 353, 326, 2142, 434, 890, 471, 1381, 35, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [1, 536, 12439, 13332, 327, 890, 397, 1381, 2, 0,

In [4]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=10,
)

print("Training arguments set up successfully!")

/Users/luddesmac/Desktop/Skola/TUM/Next-gen-prog/berlin/.venv/lib/python3.9/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Training arguments set up successfully!


In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainer

# Load model
model = AutoModelForSeq2SeqLM.from_pretrained("Salesforce/codet5-base")

# Initialize trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
)

print("Trainer initialized!")

/var/folders/6l/7_0msgkd17x1_1b_v3b4zlbc0000gn/T/ipykernel_72205/1990574301.py:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Trainer initialized!


In [6]:
# Start training
trainer.train()

print("Model training complete!")

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.000000,nan
2,0.000000,nan
3,0.000000,nan


Model training complete!


In [7]:
# Save trained model and tokenizer
model.save_pretrained("./trained_codet5")
tokenizer.save_pretrained("./trained_codet5")

print("Model saved locally!")

Model saved locally!


In [8]:
# Reload model for testing
model = AutoModelForSeq2SeqLM.from_pretrained("./trained_codet5")
tokenizer = AutoTokenizer.from_pretrained("./trained_codet5")

# Function to generate code
def generate_code(question, model, tokenizer, max_length=200):
    inputs = tokenizer(question, return_tensors="pt")
    outputs = model.generate(inputs["input_ids"], max_length=max_length)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test model with a sample question
test_question = "Convert this word problem into Python code: What is the sum of 7 and 5?"
generated_code = generate_code(test_question, model, tokenizer)

print("Test Question:", test_question)
print("Generated Code:", generated_code)

Test Question: Convert this word problem into Python code: What is the sum of 7 and 5?
Generated Code: 
